# Arl_Recommender

---------

## İş Problemi

**Senaryo**

Aşağıda 3 farklı kullanıcının sepet bilgileri verilmiştir. Bu sepet bilgilerine en
uygun ürün önerisini birliktelik kuralı kullanarak yapınız. Ürün önerileri 1 tane
ya da 1'den fazla olabilir. Karar kurallarını 2010-2011 Germany müşterileri
üzerinden türetiniz.
Kullanıcı 1’in sepetinde bulunan ürünün id'si: 21987
Kullanıcı 2’in sepetinde bulunan ürünün id'si : 23235
Kullanıcı 3’in sepetinde bulunan ürünün id'si : 22747

## Veri Seti Hikayesi

*Online Retail II isimli veri seti İngiltere merkezli bir perakende şirketinin 01/12/2009 - 09/12/2011 tarihleri arasındaki online satış
işlemlerini içeriyor. Şirketin ürün kataloğunda hediyelik eşyalar yer almaktadır ve çoğu müşterisinin toptancı olduğu bilgisi
mevcuttur.*

In [ ]:
### Değişkenler

##### InvoiceNo : Fatura Numarası
##### StockCode : Ürün kodu
##### Description : Ürün ismi
##### Quantity : Ürün adedi 
##### InvoiceDate : Fatura fiyatı
##### UnitPrice : Fatura fiyatı ( Sterlin )
##### CustomerID : Eşsiz müşteri numarası
##### Country : Ülke ismi 

## To do List:

**GÖREV 1**: Veriyi Hazırlama

**GÖREV 2**: Alman Müşteriler Üzerinden Birliktelik Kuralları Üretme

**GÖREV 3**: Sepet İçerisindeki Ürün Id’leri Verilen Kullanıcılara Ürün Önerisinde Bulunma



-------

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Pandas kütüphanesini import edelim;
import pandas as pd
import numpy as np

# pandas görüntü ayarlarını yapalım;

# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.float_format', lambda x: '%.3f' % x)

# veri setini dahil edelim;

df_ = pd.read_excel(r"/content/drive/MyDrive/recommendation_cases/bonus_arl_3/online_retail_II.xlsx",sheet_name = "Year 2010-2011")
df = df_.copy()
df.head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


### Veriyi Hazırlama

In [45]:
# Retail veri setinin ayarlamalarını yapalım;

def outlier_thresholds(df,col):
  q1 = df[col].quantile(.01)
  q3 = df[col].quantile(.99)

  QIR = q3-q1
  lower = q1 - QIR*1.5
  upper = q3 + QIR*1.5

  return lower,upper

def replace_outlier_thresholds(df,col):
  lower,upper = outlier_thresholds(df,col)

  df.loc [(df[col] >= upper) , col] = upper
  df.loc [(df[col] <= lower) , col] = lower 

  return df

def retail_data_prep(df):
  # "POST" olan gözlemleri çıkaralım;
  df = df[~df["StockCode"].isin(["POST"])]
  # "C" ile başlayan iade gözlemleri çıkaralım;
  df = df[~(df["Invoice"].str.contains("C",na=False))]
  # Boş gözlemleri drop edelim;
  df.dropna(inplace=True)
  # Price Değeri Sıfırdan küçük olanları çıkaralım;
  df = df[df["Price"]>0]
  # aykırı değerleri baskılayalım;
  df = replace_outlier_thresholds(df,"Price")
  df = replace_outlier_thresholds(df,"Quantity")

  return df





In [52]:
# Germany ölçekte analiz yapmak için veri setini germany olarak filtreleyelim;

df_gr = df[df["Country"] == "Germany"]

# veri setinin ön işlemesini tamamlayalım;
df_gr = retail_data_prep(df_gr)

# değişkenlerin dağılımlarını inceyeleim;

df_gr.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,8666.0,13.539407,16.014961,1.00,6.00,12.00,12.00,238.50
Price,8666.0,2.744201,2.928551,0.04,1.25,1.65,3.25,31.56
Customer ID,8666.0,12649.241634,314.326226,12426.00,12480.00,12600.00,12662.00,14335.00


### Alman Müşteriler Üzerinden Birliktelik Kuralları Üretme

In [54]:
# veri setini arl için uygun hale getirelim;

pvt = df_gr.groupby(["Invoice","StockCode"])["Quantity"].sum().unstack().fillna(0).applymap(lambda x : 1 if x>0 else 0)

pvt.head()

StockCode,10002,10125,10135,11001,15034,15036,15039,16008,16011,16014,...,90160D,90161B,90161C,90161D,90201A,90201B,90201C,90201D,90202D,M
Invoice,,,,,,,,,,,,,,,,,,,,,
536527,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536840,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536861,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536967,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536983,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
# kullanılacak kütüphaneleri import edelim;

from mlxtend.frequent_patterns import apriori,association_rules

# kurallar ve algoritamalar için fonksiyon oluşturalım;

def create_rules(pvt):
  # apriroi algoritması 
  df_apr = apriori(pvt,min_support = 0.01 , use_colnames = True)
  # birliktelik kuralının oluşturulması
  rules = association_rules(df_apr,metric = "support" , min_threshold=0.01)
  return rules 

rules = create_rules(pvt)

rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(16237),(22326),0.011136,0.249443,0.011136,1.000000,4.008929,0.008358,inf
1,(22326),(16237),0.249443,0.011136,0.011136,0.044643,4.008929,0.008358,1.035073
2,(20674),(20675),0.022272,0.033408,0.013363,0.600000,17.960000,0.012619,2.416481
3,(20675),(20674),0.033408,0.022272,0.013363,0.400000,17.960000,0.012619,1.629547
4,(20674),(20676),0.022272,0.037862,0.011136,0.500000,13.205882,0.010293,1.924276


### Sepet İçerisindeki Ürün Id’leri Verilen Kullanıcılara Ürün Önerisinde Bulunma

In [59]:
# ürün kodlarını isimlerine çeviren bir fonksiyon yazalım;
def check_id(code):
  name = df[df["StockCode"] == code].iloc [0][["Description"]][0]
  return name

  

In [76]:
# arl_recommender fonksiyonu ile output verecek kısmı hazırlayalım;

def arl_recommender(rules,products_id,r_count = 1):
  recom_list = []
  rules = rules.sort_values("lift",ascending=False)
  for index,products in enumerate(list(rules["antecedents"])):
    for k in list(products):
      if k == products_id:
          log = list(rules.iloc [index]["consequents"])[0]
          if log not in recom_list:
            recom_list.append(log)
  
  return recom_list[:r_count]



In [81]:
# gelen ürün kodlarını bir rapor haline getiren bir fonksiyon hazırlayalım;

def check_product(code,r_coder = 3):
  liste = arl_recommender(rules,code,r_count = r_coder)
  sayac = 0

  print(f"Satın Alınan ÜRÜN : {check_id(code)}\n","-"*30,sep="")
  
  for i in liste:
    sayac += 1 
    names = check_id(i)
    print(f"{sayac}. ürün tavsiyesi {check_id(i)}")


In [82]:
# Kullanıcı 1’in sepetinde bulunan ürünün id'si: 21987

check_product(21987)

Satın Alınan ÜRÜN : PACK OF 6 SKULL PAPER CUPS
------------------------------
1. ürün tavsiyesi PACK OF 20 SKULL PAPER NAPKINS
2. ürün tavsiyesi PACK OF 6 SKULL PAPER PLATES
3. ürün tavsiyesi SET/6 RED SPOTTY PAPER CUPS


In [83]:
# Kullanıcı 2’in sepetinde bulunan ürünün id'si : 23235

check_product(23235, 5)

Satın Alınan ÜRÜN : STORAGE TIN VINTAGE LEAF
------------------------------
1. ürün tavsiyesi ROUND STORAGE TIN VINTAGE LEAF
2. ürün tavsiyesi SET OF TEA COFFEE SUGAR TINS PANTRY
3. ürün tavsiyesi SET OF 4 KNICK KNACK TINS DOILEY 
4. ürün tavsiyesi DOILEY STORAGE TIN
5. ürün tavsiyesi SET OF 4 KNICK KNACK TINS LEAVES 


In [85]:
# Kullanıcı 3’in sepetinde bulunan ürünün id'si : 22747

check_product(22747,3)

Satın Alınan ÜRÜN : POPPY'S PLAYHOUSE BATHROOM
------------------------------
1. ürün tavsiyesi POPPY'S PLAYHOUSE LIVINGROOM 
2. ürün tavsiyesi POPPY'S PLAYHOUSE BEDROOM 
3. ürün tavsiyesi POPPY'S PLAYHOUSE KITCHEN
